# Code Walkthrough

This notebook describes the purpose and structure of the code provided

This code was originally written for users wishing to get a better view of the quality, structure and relationships present within their various data sources. This would allow them to fix data quality issues, ensure GDPR compliance and better extract untapped insight from their data assets.

This package facilitates the implementation of custom dashboards displaying custom metrics on a variety of data sources. The backend is organised into logical objects (data sources, metrics, dashboards etc) which allows decoupling of different data sources/connections, metric definitions, UIs and processing. 

For example you can show on one dashboard a selection of metrics all derived from different data sources, some of which are computed in memory, others where the processing is delegated to a SQL database server, all with a clean coding interface. It is also easy to add new bespoke metrics to the package. 

As well as implementing the backend, this code also implements an indicative frontend for one particular realisation of a dashboard.

The project is a work in progress b

## Running the demo frontend

To run, clone the package, install the requirements, ensure the example data is saved into the app/files folder and run python runserver.py

## Backend Walkthrough

### Summary
The core classes are:
- Tabular Readers
To read various data formats (files or SQL queries) into in-memory dataframes
- SQL connectors
To connect to relational databases (any dialect)*
- Data Sources
Contains a data source and defines compute location
- Metrics
Functional code for various calculations. Also stores previously computed results
- Dashboards
Object conatining a collection of metrics and other dashboard-state data 

These backend classes can be found in the app/profiler folder of the repo.

*NB all SQL functionality is omitted from the demo so it can be simply run in other environments

### Tabular Readers and Data Sources

In [17]:
import warnings
warnings.filterwarnings('ignore')
from app.profiler.tabular_readers import CSVReader, JSONReader, ExcelReader, DataFrameReader

csv_reader = CSVReader(r'app/files','demo_csv.csv')
json_reader = JSONReader(r'app/files','demo_json.json')
excel_reader = ExcelReader(r'app/files','demo_xlsx.xlsx')

csv_reader.get_data()

,a,b,c
0,1,4,cat
1,2,5,dog
2,3,6,apple


In [2]:
json_reader.get_data()

,a,b,c
0,1,4,cat
1,2,5,dog
2,3,6,apple


In [3]:
excel_reader.get_data()

,a,b,c
0,1,4,cat
1,2,5,dog
2,3,6,apple


Another supported version for the above is a SQL table or query that returns tabular data small enough to fit in memory.

In memory data sources take tabular data as an argument and are initialised as below:

In [4]:
from app.profiler.data_sources import InMemoryDataSource

In [5]:
ds = InMemoryDataSource(csv_reader)
ds.get_column_names()

Index(['a', 'b', 'c'], dtype='object')

The underlying source for a SQLDataSource is a SQLViewConnector rather than TabularReader

### Metrics

Metrics are defined by 2 things - a calculation definition and a data source. They can be initialised with all necessary parameters without being calculated, then their calculation code can be run on demand. Once run, the result is stored as an attribute. Note the actual computation happens as dictated by the data source (either in memory or on the DB server). When called, the metric passes the calculation code to the data source, which then executes it on its data in the desired environment. 

In [6]:
from app.profiler.metrics import *

In [7]:
# Initialise metric with data source
demo_data_tot_rows_cols = TotalRowsCols(ds)
# Call metric to run calculation
demo_data_tot_rows_cols()
# Each time result is needed, retrieve from result attribute
rows,cols = demo_data_tot_rows_cols.get_result()
rows,cols

(3, 3)

If the function requires parameters, these can be passed in on instatiation:

In [8]:
# new data table for this example
csv2 = CSVReader(r'app/files','demo_csv_2.csv')
csv2.get_data()

,a,b,c
0,1.0,NaN,cat
1,2.0,5.0,dog
2,NaN,6.0,NaN


In [9]:
ds2 = InMemoryDataSource(csv2)
# Initialise metric 
demo_data_blank_cells_as_pc = TotalBlankCells(ds2, {'pc':True})
demo_data_blank_cells = TotalBlankCells(ds2, {'pc':False})
# calculate on demand
demo_data_blank_cells()
demo_data_blank_cells_as_pc()
# retrieve result as %
demo_data_blank_cells_as_pc.get_result()

0.33

In [10]:
# retrieve result as total number
demo_data_blank_cells.get_result()

3

In [11]:
# function parameters can also be changed at calculation time
demo_data_blank_cells(pc=True)

0.33

Metric code can also be called as a static method for one-off calculation

In [12]:
import pandas as pd

df = pd.DataFrame({
                "symbol": ["A", "B", "C", "A", "B", "C"],
                "price": [12, 24, 48, 14, 13, 20],
            })
df


,symbol,price
0,A,12
1,B,24
2,C,48
3,A,14
4,B,13
5,C,20


In [13]:
BasicProfile.calculate_in_mem(df, incl_graph=False)

,01. Column Name,02. Data Type,03. Row Count,04. Nulls,05. Non-Nulls,06. No. Unique Values,07. Average Value,08. Standard Deviation,09. Minimum,10. Maximum,11. 25%,12. 50%,13. 75%,count
0,symbol,object,6,0,6,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,price,int64,6,0,6,6,21.833333,13.629625,12.0,48.0,13.25,17.0,23.0,6.0


In [14]:
TotalRowsCols.calculate_in_mem(df)

(6, 2)

In [15]:
TotalBlankCells.calculate_in_mem(df, pc=True)

0.0

More complex and bespoke metrics can be easily applied as above. For example the below imports a pretrained ML model to extract valid string but invalid addresses from the specified column:

In [19]:
df = pd.DataFrame({
                "user_id": [1, 2, 3],
                "address": ['4 Mill Lane, London', '56 Cowper Street, Edinburgh', 'No address found here'],
            })
ds = InMemoryDataSource(DataFrameReader(df))

demo_bad_address_metric = DetectBadAddress(ds, {'id_col':'user_id','address_col':['address']})
demo_bad_address_metric()
ans = demo_bad_address_metric.get_result()
# print those with validity score < 50%
ans.loc[ans["Validity Score"] < 0.5]

,user_id,addr,Validity Score
2,3,No address found here,0.32


It's easy to add a new metric to the system:

In [23]:
class AddOnes(Metric):
    def __init__(self, data_source, metric_args={}):
        super().__init__(data_source, metric_args)
        
    @staticmethod
    def calculate_in_mem(inp):
        inp['ones'] = 1
        return inp
    

In [24]:
demo_add_ones = AddOnes(ds)
demo_add_ones()

,user_id,address,ones
0,1,"4 Mill Lane, London",1
1,2,"56 Cowper Street, Edinburgh",1
2,3,No address found here,1


### Dashboards

Dashboards are objects containing a list of metrics and other dashboard state information. They should contain all the data needed for a frontend to simply call and manipulate as needed on the UI.

In [25]:
from app.profiler.dashboards import *

demo_data_source = InMemoryDataSource(CSVReader(r'app/files','demo_csv.csv'))
custom_dash = Dashboard()
custom_dash.add_metrics([BasicProfile(demo_data_source,{'incl_graph':False}),TotalRowsCols(demo_data_source),TotalBlankCells(demo_data_source,{'pc':True})])
custom_dash.calculate_dashboard()
custom_dash.get_all_results()

[  01. Column Name 02. Data Type  03. Row Count  04. Nulls  05. Non-Nulls  \
 0               a         int64              3          0              3   
 1               b         int64              3          0              3   
 2               c        object              3          0              3   
 
    06. No. Unique Values  07. Average Value  08. Standard Deviation  \
 0                      3                2.0                     1.0   
 1                      3                5.0                     1.0   
 2                      3                NaN                     NaN   
 
    09. Minimum  10. Maximum  11. 25%  12. 50%  13. 75%  count  
 0          1.0          3.0      1.5      2.0      2.5    3.0  
 1          4.0          6.0      4.5      5.0      5.5    3.0  
 2          NaN          NaN      NaN      NaN      NaN    NaN  ,
 (3, 3),
 0.0]

Dashboards therefore don't depend on data sources directly, so it is easy to view metrics calculated from different sources in one view. Likewise, custom metrics can be integrated, underlying file structures can be swapped out and compute can be delegated with minimal impact on the codebase.



# Example Frontend

To see a more fleshed-out version of a dasboard with UI implementation, do the following:

1. clone this repo
2. create  and activate virtual environment (`python3 -m venv venv && source venv/bin/activate`)
3. `pip install -r requirements.txt`
4. make sure mortgage_data_v4.csv is saved in app/files directory
5. `python runserver.py` - the dashboard should now be available at http://localhost:8080

Click through as prompted to see the results.

If this is not possible, screenshots are available in the README.md file

# Run Tests

To run the unit tests, use the shell script ./run_tests.sh, remembering to allow execute permissions first (`chmod +x run_tests.sh`)